In [2]:
import numpy as np
from scipy import optimize
import math

In [8]:
r = np.zeros(c.size).astype(int)
for i in range(c.size):
    s = np.array(r)
    s = np.vstack((s, s))
    A = np.vstack((A, s))

r = np.zeros(2*c.size).astype(int)
b = np.hstack((b, r))

R = np.array([0])
R = BranchAndBound(-c, A, b, 0, 1, R, k)
# print(R)
max(-R)


295

In [6]:
erro = 0.01


def Esquerda(c, A_e, b_e, l, u, S, sol, r, k):
    A_e[k + 2*r - 1][r - 1] = 1
    # Muda a restricao no vetor b
    b_e[k + 2*r - 1] = math.floor(sol[r - 1])

    S = BranchAndBound(c, A_e, b_e, l, u, S, k)
    return S


def Direita(c, A_d, b_d, l, u, S, sol, r, k):
    A_d[k + 2*r][r - 1] = -1
    # Muda a restricao no vetor b
    b_d[k + 2*r] = -math.ceil(sol[r - 1])

    S = BranchAndBound(c, A_d, b_d, l, u, S, k)
    return S


def BranchAndBound(c, Aub, bub, l, u, S, k):
    # Resolver o problema por simplex
    Sol = optimize.linprog(c=c, A_ub=Aub, b_ub=bub,
                           bounds=(l, u), method='simplex')

    # Ver se é factível
    v = 0
    for i in range(k+1):
        if np.inner(A[i, :], Sol.x) > b[i]:
            v = 1
    if v == 1:
        return S

    # Ver se a solução é inteira
    if np.linalg.norm(abs(Sol.x - np.array(Sol.x).astype(int))) <= erro:
        S = np.insert(S, S.size, Sol.fun)

        return S

    else:
        # Ver qual variavel tem a maior parte fracionaria
        r = abs(Sol.x - np.array(Sol.x).astype(int)).tolist()
        r = r.index(max(r)) + 1

        # Filho da esquerda
        S = Esquerda(c, np.array(Aub), np.array(
            bub), l, u, S, np.array(Sol.x), r, k)

        # Filho da Direita
        S = Direita(c, np.array(Aub), np.array(bub),
                    l, u, S, np.array(Sol.x), r, k)

        return S
